# Add output variable in CAM


<div class="alert alert-info">
<strong>Exercise: Add an additional output variable</strong><br><br>
    
Create a case called `b1850_T750` using the compset `B1850` at `f19_g17` resolution. 
- Add an output field for the temperature at 750 mbar. 
- Output daily values of `T750` and `T500` in the `h1` history file. 
- Set the namelist to output a single `h1` for the run. 
- Make a 1-month run. 

</div>


<div class="alert alert-warning">  
<details> 
   <summary>
      <font face="Times New Roman" color='blue'>Click here for hints</font>
    </summary>
    
**Tip to add `T750`**  <br>
- Use `T500` as a template for your changes.

- Find the subroutine containing `T750`. 
For instance, in the CESMROOT, use the command: 

```
    grep –r T500 *    
```
    
    
**Tip to check your solution `T750`** <br>
- When the run is completed, go to your archive directory:
- check the fields `T750` and `T500` are in the file `h1`
- create a file with the difference between `T750-T500`
- For instance, you can use ``ncap2`` 
```
    ncap2 -s ’T750_minus_T500=T750-T500' b1850_T750.cam.h1.0001-01-01-00000.nc  T750-T500.nc
```
- Look at the difference with ``ncview``. 
```
    ncview T750-T500.nc
```
</details>
</div>



<div class="alert alert-success">   
<details>
<summary><font face="Times New Roman" color='blue'>Click here for the solution</font></summary><br>
 
**# Create a new case**
    
Create a new case `b1850_T750` with the command:
```
 cd /glade/work/$USER/code/my_cesm_code/cime/scripts/
./create_newcase --case ~/cases/b1850_T750 --compset B1850 --res f19_g17 
```

**# Setup**

Invoke case.setup with the command:
```    
cd ~/cases/b1850_T750
./case.setup
```

**# Make Source Modifications**    
Use `T500` as a template for your changes. For that purpose, locate the file where `T500` is computed and copy it into
    `SourceMods/src.atm`:
    
```    
cp /glade/campaign/cesm/cesm/tutorial/cesm2.1_tutorial2022/components/cam/src/physics/cam/cam_diagnostics.F90 SourceMods/src.cam
```

Now, let's use `T500` as a template for your changes and add the relevant lines for `T750`: edit the file `SourceMods/src.cam/cam_diagnostics.F90` to add the following.
    
**First change**
    
Under the lines: 
```
!++ add a variable for T500 
call addfld ('T500', horiz_only,  'A', 'K','Temperature at 500 mbar pressure surface') 
```
add the lines:

```
!++ add a variable for T750 
call addfld ('T750', horiz_only,  'A', 'K','Temperature at 750 mbar pressure surface')  
```
    
**Second change**
Under the lines: 
```
!++ add a variable for T500 
if (hist_fld_active('T500')) then 
    call vertinterp(ncol, pcols, pver, state%pmid, 50000._r8, state%t, p_surf, & 
         extrapolate='T', ps=state%ps, phis=state%phis)
    call outfld('T500    ', p_surf, pcols, lchnk )
end if
```
add the lines:
    
```    
!++ add a variable for T750 
if (hist_fld_active('T750')) then 
    call vertinterp(ncol, pcols, pver, state%pmid, 75000._r8, state%t, p_surf, & 
         extrapolate='T', ps=state%ps, phis=state%phis)
    call outfld('T750    ', p_surf, pcols, lchnk )
end if
```

**# Customize namelists**
Edit the file `user_nl_cam` and add the lines:
```
    nhtfrq = 0, -24
    mfilt = 1, 31
    fincl2 = 'T750', 'T500'
```
    
    
**# Set run length**

Set the **run length** to 1 month:
```   
./xmlchange STOP_N=1,STOP_OPTION=nmonths
```

**# Change the job queue and account number**

If needed, change `job queue` and `account number`. <br>
For instance, to run in the queue `regular` and the project number `P93300642`, use the command:

```  
./xmlchange JOB_QUEUE=regular,PROJECT=P93300642
```

**# Build and submit**
    
Build the model and submit your job:
```
qcmd -- ./case.build
./case.submit
```
____
    
**# Look at your solution**
    
When the run is completed, check the fields `T750` and `T500` are in the file `h1`:
```  
cd /glade/derecho/scratch/$USER/archive/b1850_T750/atm/hist/ 
ncdump -h b1850_T750.cam.h1.0001-01-01-00000.nc
```
    
The file should contain:
```
float T500(time, lat, lon) ;
    T500:units = "K" ;
    T500:long_name = "Temperature at 500 mbar pressure surface" ;
    T500:cell_methods = "time: mean" ;
float T750(time, lat, lon) ;
    T750:units = "K" ;
    T750:long_name = "Temperature at 750 mbar pressure surface" ;
    T750:cell_methods = "time: mean" ;
```
If you don't see these variables, check you correctly set the `user_nl_cam`.
    
Create a file with the difference between `T750-T500`:
```    
cd /glade/derecho/scratch/$USER/archive/b1850_T750/atm/hist/ 
ncap2 -s 'T750_minus_T500=T750-T500' b1850_T750.cam.h1.0001-01-01-00000.nc  T750-T500.nc 
```

Look at the difference between `T750-T500` with `ncview`: 
```
cd /glade/derecho/scratch/$USER/archive/b1850_T750/atm/hist/ 
ncview T750-T500.nc
```
  
The field `T750-T500` looks like:

*<p style="text-align: center;"> 
    ![ncview T750-T500](../../../images/sourcemods/ncview_T750-T500.png)  
    Figure: Overview of the CESM directories and the SourceMods directories. </p>*
    
</details>
</div>

